## Homework 5

In this week's class, we went through the Recsys Chanllege 2015: \
https://2015.recsyschallenge.com/challenge.html\
For this homework, we will work on task 1. So, when you create features, you will need to think on the session level but not item level.

The click and buy datasets are unploaded to iCollege. These 2 files are sampled down to ~50k buy and ~50k not buy sessions to simplify your homework.

In this homework, please do feature engineering to create new features from the click and buy datasets. You can use the ideas I provided in the class but I encourage you to be creative and think on your own as well. 

Each feautre you creat will worth 10 points and the maximal points from feature engineerring will cap at 80 points. But do not limit yourself to 8 features because the more features you can create, the better they will help you in your next homework(6) for Machine Learning modeling.

In the end, you will need to create the Analytics Base Table(ABT) which worth 20 points. In the ABT, you should have each row representing a unique click session but not each session with each item. This is different from my code on github because we are only doing task 1 for this homework.

In [1]:
# loading data
import pandas as pd

click = pd.read_csv('click_sep.csv', low_memory=False)

buy = pd.read_csv('buy_sep.csv',low_memory=False)

In [2]:
click.head()

,SessionID,TimeStamp,ItemID,Category
0,9293568,2014-09-01 18:07:00.855000+00:00,214853225,S
1,9293653,2014-09-01 10:38:47.087000+00:00,214834871,S
2,9293653,2014-09-01 10:39:49.115000+00:00,214849327,S
3,9293653,2014-09-01 10:40:31.736000+00:00,214828970,S
4,9293653,2014-09-01 10:41:01.640000+00:00,214849327,S


In [3]:
print('unique click session')
print(len(click.SessionID.unique()))

unique click session
99998


In [4]:
buy.head()

,SessionID,TimeStamp,ItemID,Price,Quantity
0,9641594,2014-09-01 09:09:25.575000+00:00,214853342,2093,2
1,9641594,2014-09-01 09:09:25.596000+00:00,214853340,837,2
2,9641594,2014-09-01 09:09:25.614000+00:00,214853420,1046,2
3,9431393,2014-09-01 13:38:48.351000+00:00,214846258,941,1
4,9431393,2014-09-01 13:38:48.414000+00:00,214853340,837,1


In [5]:
print('unique buy session')
print(len(buy.SessionID.unique()))

unique buy session
50000


 ## Examples for feature engineering
 
 1 point for explaining each feature.
 
 You will not get this 1 point if you do not have code for it, which means only the idea will not count for points.
 
 If your coding is incorrect, you still get 1 point for explaining the idea and trying to code for it.
 
 Coding without this explaination will result in 0 points for this feature.

### Example 1 - features from click dataset only
you can create a new dataframe for each feature then join by session ID in the end in your ABT

In [6]:
# feature 1 
# In each click session, how many items were clicked                                                    ---- 1 point

feature1 = click.groupby('SessionID')['ItemID'].count().reset_index(name="item_cnt_in_ses")            #---- 9 points

In [7]:
feature1.head()

,SessionID,item_cnt_in_ses
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [8]:
len(feature1)

99998

### Example 2 - features from click dataset only
you can directly create new column for your feature in the original dataframe, 
then only select the columns you need in the end in your ABT.

In [9]:
# feature 2 
# In which weekday the click activity happened.                                                          ---- 1 point

import datetime
click['click_weekday'] = pd.DatetimeIndex(click['TimeStamp']).dayofweek                                #---- 9 points

In [10]:
# this step is to investigate duplicates 
feature2 = click[['SessionID','click_weekday']].drop_duplicates()
feature2.head()

,SessionID,click_weekday
0,9293568,0
1,9293653,0
6,9293662,0
8,9293704,0
10,9293797,0


In [11]:
len(feature2) 

100076

There are more feature2 than feature1 because some long sessions might have clicks from 2 different weekdays, 
especailly if the session happend around midnights.
This should not be treated as duplicates but use the same session twice representing differnt weekdays' clicks.

## Example 3 -- features from click and buy dataset joined

In [12]:
#feature3 
# The average price of all items clicked in each click session                                       ---- 1 point

# step 1 join certain columns from click and buy tables                                              ---- 9 points
df = click.merge(buy[['ItemID','Price']], how = 'inner', on = 'ItemID').drop_duplicates()
print(df.head)
# this line might take some time to run, be patient.

# step 2 caculate average price from items clicked in each session
feature3 = df.groupby('SessionID')['Price'].mean().reset_index(name="ave_item_price")
print(feature3.head())



<bound method NDFrame.head of            SessionID                         TimeStamp     ItemID Category  \
0            9293568  2014-09-01 18:07:00.855000+00:00  214853225        S   
7            9293568  2014-09-01 18:07:00.855000+00:00  214853225        S   
22           9378348  2014-09-01 20:59:36.817000+00:00  214853225        S   
29           9378348  2014-09-01 20:59:36.817000+00:00  214853225        S   
44           9354957  2014-09-01 09:57:12.293000+00:00  214853225        S   
...              ...                               ...        ...      ...   
142184501   11292929  2014-09-27 11:38:11.837000+00:00  214554967        5   
142184502   11289399  2014-09-24 17:01:18.671000+00:00  214551294        8   
142184503   11299968  2014-09-27 18:41:26.992000+00:00  214769111       11   
142184504   11299968  2014-09-27 18:42:18.887000+00:00  214769589       11   
142184505   11298832  2014-09-27 17:28:44.687000+00:00  214835199        9   

           click_weekday  Price  

In [13]:
len(feature3)

96580

This feature will have missing values because the downsample caused some items are not in buy table anymore, therefore you won't find the price for the item.

In [14]:
# feature 4
# explain your feature: The total number of items bought by a customer
feature4 = buy.groupby('SessionID')['Quantity'].count().reset_index(name="amount_bought")

In [15]:
feature4.head()

,SessionID,amount_bought
0,9195209,1
1,9195293,2
2,9195434,1
3,9195476,1
4,9195828,3


In [16]:
# feature 5
# explain your feature: Whether the session is a repeat buyer, 1 if the session has purchased previously, 0 if haven't purchased before
buy['buy_date'] = pd.DatetimeIndex(buy['TimeStamp']).date
temp = buy[['SessionID','buy_date']].drop_duplicates()

In [17]:
repeat_buyer_list = temp[temp.duplicated(['SessionID'])].SessionID.to_list()

In [18]:
buy['repeat_buyer'] = buy.apply(lambda x: 1 if x['SessionID'] in repeat_buyer_list else 0, axis=1)

In [19]:
feature5 = buy[['SessionID','repeat_buyer']].drop_duplicates()
feature5.head()

,SessionID,repeat_buyer
0,9641594,0
3,9431393,0
6,9431411,0
9,9725713,0
16,9293662,0


In [20]:
# feature 6
# explain your feature: The total amount of money spent by the same session
buy['total_sale'] = buy.apply(lambda x: x['Price'] * x['Quantity'], axis=1)
feature6 = buy.groupby('SessionID')['total_sale'].sum().reset_index(name='total_spent')


In [21]:
feature6.head()

,SessionID,total_spent
0,9195209,16650
1,9195293,3138
2,9195434,1360
3,9195476,7328
4,9195828,5337


In [22]:
# feature 7
# explain your feature: The hour that the session usually browses

click['click_hour'] = pd.DatetimeIndex(click['TimeStamp']).hour                               
# this step is to investigate duplicates 
feature7 = click[['SessionID','click_hour']].drop_duplicates()
feature7.head()

,SessionID,click_hour
0,9293568,18
1,9293653,10
6,9293662,6
7,9293662,7
8,9293704,18


In [23]:
# feature 8
# explain your feature: The hour that the session actually makes the purchase
buy['buy_hour'] = pd.DatetimeIndex(buy['TimeStamp']).hour                           
feature8 = buy[['SessionID','buy_hour']].drop_duplicates()
feature8.head()


,SessionID,buy_hour
0,9641594,9
3,9431393,13
6,9431411,11
9,9725713,14
16,9293662,7


In [24]:
# feature 9
# explain your feature: The amount of clicks a user does for a particular category
feature9 = click.groupby(['SessionID'])['Category'].value_counts().reset_index(name='CategoryCount')
feature9

,SessionID,Category,CategoryCount
0,9194351,S,5
1,9194429,S,2
2,9194513,S,3
3,9194692,S,1
4,9194824,1,1
...,...,...,...
136814,11561999,S,1
136815,11562071,S,5
136816,11562104,5,1
136817,11562107,S,3


In [25]:
# feature 10
# explain your feature: Whether the session is a repeat visitor
click['visit_date'] = pd.DatetimeIndex(click['TimeStamp']).date
temp = click[['SessionID','visit_date']].drop_duplicates()


In [26]:
repeat_visitor_list = temp[temp.duplicated(['SessionID'])].SessionID.to_list()

In [27]:
click['repeat_visitor'] = click.apply(lambda x: 1 if x['SessionID'] in repeat_visitor_list else 0, axis=1)

In [28]:
feature10 = click[['SessionID','repeat_visitor']].drop_duplicates()
feature10.head()

,SessionID,repeat_visitor
0,9293568,0
1,9293653,0
6,9293662,0
8,9293704,0
10,9293797,0


In [29]:
# feature 11
# explain your feature: The cheapest item bought by a customer during a session
feature11 = buy.groupby(['SessionID'])['Price'].min().reset_index(name='cheapest_item_price')


In [30]:
#...

### Final Step 
Create ABT on session level
### Construct Predictive Target column  ---- 10 points:
If there is a record in buy session, then it means the buy activitiy happened and that buy session should be marked as 1.
In click sessions, there are many session ids not in the buy table, that means the click activities happened but buy activity did not happen, then these click sessions should be marked as 0.

In [31]:
buy['hasBought'] = 1

In [32]:
ABT = click['SessionID'].drop_duplicates()

In [33]:
ABT = pd.merge(ABT,buy[['SessionID','hasBought']],on='SessionID',how='left')

In [34]:
ABT['hasBought'] = ABT.apply(lambda x: x['hasBought'] if x['hasBought'] == 1 else 0, axis=1)

### Join all features together with predictive target column in the final ABT ---- 10 Points

In [35]:
ABT = pd.merge(ABT,feature1,on='SessionID',how='left')
ABT = pd.merge(ABT,feature2,on='SessionID',how='left')
ABT = pd.merge(ABT,feature3,on='SessionID',how='left')
ABT = pd.merge(ABT,feature4,on='SessionID',how='left')
ABT = pd.merge(ABT,feature5,on='SessionID',how='left')
ABT = pd.merge(ABT,feature6,on='SessionID',how='left')
ABT = pd.merge(ABT,feature7,on='SessionID',how='left')
ABT = pd.merge(ABT,feature8,on='SessionID',how='left')
ABT = pd.merge(ABT,feature9,on='SessionID',how='left')
ABT = pd.merge(ABT,feature10,on='SessionID',how='left')
ABT = pd.merge(ABT,feature11,on='SessionID',how='left')

In [36]:
ABT

,SessionID,hasBought,item_cnt_in_ses,click_weekday,ave_item_price,amount_bought,repeat_buyer,total_spent,click_hour,buy_hour,Category,CategoryCount,repeat_visitor,cheapest_item_price
0,9293568,0.0,1,0,653.5,NaN,NaN,NaN,18,NaN,S,1,0,NaN
1,9293653,0.0,5,0,5706.0,NaN,NaN,NaN,10,NaN,S,4,0,NaN
2,9293653,0.0,5,0,5706.0,NaN,NaN,NaN,10,NaN,1,1,0,NaN
3,9293662,1.0,2,0,1151.0,1.0,0.0,1151.0,6,7.0,S,2,0,1151.0
4,9293662,1.0,2,0,1151.0,1.0,0.0,1151.0,7,7.0,S,2,0,1151.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383597,11299794,1.0,10,2,1360.3,2.0,0.0,7328.0,19,19.0,1,1,0,3664.0
383598,11299793,1.0,3,3,1465.0,2.0,0.0,3663.0,7,7.0,S,2,0,1570.0
383599,11299793,1.0,3,3,1465.0,2.0,0.0,3663.0,7,7.0,5,1,0,1570.0
383600,11299793,1.0,3,3,1465.0,2.0,0.0,3663.0,7,7.0,S,2,0,1570.0


In [37]:
ABT.to_csv('ABT.csv')